Based on https://github.com/James-Jagielski/7-bit-D-A-converter-MP4-MADVLSI/blob/main/schematics/simulations/plot_dac_mc.ipynb with permission of repo owners

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from si_prefix import si_format

In [4]:
n_bits = 7
mc_simulations = []
nano = 1e-9
results_dir = "./mc"

In [6]:
for filename in os.listdir(results_dir):
    df = pd.read_csv(os.path.join(results_dir, filename), delimiter = "\s+")
    # print(df.head(3))
    print(df)
    df["code"] = df["v0"] * 0 # init column to all 0s
    for bit in range(n_bits):
        # df["code"] = df["code"] + (df[f"v{bit}"] > .5*df["vdd"]) * 2 ** (n_bits - bit - 1)
        df["code"] = df["code"] + (df[f"v{bit}"] > .5*df["vdd"]) * 2 ** (n_bits - bit - 1)
        df.drop(f"v{bit}", axis = "columns", inplace = True)
    df.rename({"i(viout)": "Iout"}, axis = "columns", inplace = True)

         net3  v(b0)    net3.1  v(b1)    net3.2  v(b2)    net3.3  v(b3)  \
0    0.192450    0.0  0.192450    0.0  0.192450    0.0  0.192450    0.0   
1    0.192444    1.8  0.192444    0.0  0.192444    0.0  0.192444    0.0   
2    0.192443    0.0  0.192443    1.8  0.192443    0.0  0.192443    0.0   
3    0.192440    1.8  0.192440    1.8  0.192440    0.0  0.192440    0.0   
4    0.192441    0.0  0.192441    0.0  0.192441    1.8  0.192441    0.0   
..        ...    ...       ...    ...       ...    ...       ...    ...   
123  0.192314    1.8  0.192314    1.8  0.192314    0.0  0.192314    1.8   
124  0.192316    0.0  0.192316    0.0  0.192316    1.8  0.192316    1.8   
125  0.192313    1.8  0.192313    0.0  0.192313    1.8  0.192313    1.8   
126  0.192313    0.0  0.192313    1.8  0.192313    1.8  0.192313    1.8   
127  0.192312    1.8  0.192312    1.8  0.192312    1.8  0.192312    1.8   

       net3.4  v(b4)    net3.5  v(b5)    net3.6  v(b6)    net3.7  i(v_i_in)  \
0    0.192450    0.0